
Import libraries




In [13]:
import pandas as pd
import re


Load the datasets

In [14]:

mountain_names_dataset = pd.read_csv('/content/sample_data/dataset_mountains.csv',delimiter=';', encoding='ISO-8859-1')
mountain_sentences_dataset  = pd.read_csv('/content/sample_data/dataset_sentence.csv',delimiter=';',encoding='ISO-8859-1')
print(mountain_sentences_dataset )
print(mountain_names_dataset )

                                              sentence
0    Everest stands as a beacon for climbers around...
1    The path to the summit of K2 is fraught with d...
2    Many adventurers have dreamed of conquering Ka...
3    At 8,516 meters, Lhotse towers over most other...
4     The first ascent of Makalu was achieved in 1955.
..                                                 ...
795  The icy slopes of Denali and the lush valleys ...
796  The scenic beauty of Elgon complements the vol...
797  Annapurna and Dhaulagiri tower majestically ov...
798  The cultural significance of Toubkal draws hik...
799  The stunning landscapes of Matterhorn and Weis...

[800 rows x 1 columns]
            label
0         Everest
1              K2
2   Kangchenjunga
3          Lhotse
4          Makalu
..            ...
85       Njesuthi
86    Drakensberg
87   Dufourspitze
88  Grossglockner
89      Weisshorn

[90 rows x 1 columns]


Convert the mountain names to a list

In [15]:

mountain_names_list = mountain_names_dataset['label'].tolist()
print(mountain_names_list)
print()
mountain_sentences_list = mountain_sentences_dataset['sentence'].tolist()
print(mountain_sentences_list)

['Everest', 'K2', 'Kangchenjunga', 'Lhotse', 'Makalu', 'Cho Oyu', 'Dhaulagiri', 'Manaslu', 'Nanga Parbat', 'Annapurna', 'Gasherbrum I', 'Broad Peak', 'Gasherbrum II', 'Shishapangma', 'Elbrus', 'Denali', 'Kilimanjaro', 'Aconcagua', 'Vinson', 'Wilhelm', 'Kosciuszko', 'Rainier', 'Saint Elias', 'Fuji', 'Blanc', 'Fitz Roy', 'Logan', 'Whitney', 'Roraima', 'Matterhorn', 'Pikes Peak', 'Kinabalu', 'Ararat', 'Etna', 'Toubkal', 'Olympus', 'McKinley', 'Erebus', 'Shasta', 'Hood', 'St. Helens', 'Table', 'Adams', 'Baker', 'Mauna Kea', 'Mauna Loa', 'Ngauruhoe', 'Tasman', 'Ruapehu', 'Aoraki', 'Wilson', 'Taranaki', 'Hagen', 'Batur', 'Sinabung', 'Tambora', 'Merapi', 'Semeru', 'Agung', 'Rinjani', 'Kerinci', 'Bromo', 'Fansipan', 'Elgon', 'Kenya', 'Stanley', 'Speke', 'Rwenzori', 'Karisimbi', 'Nyiragongo', 'Bisoke', 'Muhabura', 'Gahinga', 'Cameroon', 'Jebel Toubkal', 'Sinai', 'Dinali', 'Gunung', 'Mera Peak', 'Damavand', 'Zugspitze', 'Triglav', 'Khuiten', 'Belukha', 'Chersky', 'Njesuthi', 'Drakensberg', 'Dufo

Function to label a sentence in CoNLL format

Function for tokenization taking into account punctuation

In [16]:

def tokenize(sentence):
    return re.findall(r"\w+|[^\w\s]", sentence)

Function to generate labels for each sentence

In [17]:

def tag_sentence(sentence, mountain_names):
    tokens = tokenize(sentence)
    labels = ['O'] * len(tokens)

    for mountain in mountain_names:
        mountain_tokens = tokenize(mountain)
        mountain_len = len(mountain_tokens)

        for i in range(len(tokens) - mountain_len + 1):
            if tokens[i:i+mountain_len] == mountain_tokens:
                labels[i] = 'B-LOC'
                for j in range(1, mountain_len):
                    labels[i+j] = 'I-LOC'

    return tokens, labels

Create DataFrame with CoNLL format and array for data collect (Tokens, labels)


In [18]:

sentences_with_labels = []

for sentence in mountain_sentences_dataset['sentence']:
    tokens, labels = tag_sentence(sentence, mountain_names_list)
    sentence_data = {'Tokens': tokens, 'Labels': labels}
    sentences_with_labels.append(sentence_data)



First 10 sentence of database

In [19]:

for sentence_data in sentences_with_labels[:10]:
    print("Tokens:", sentence_data['Tokens'])
    print("Labels:", sentence_data['Labels'])
    print()
print(sentences_with_labels)

Tokens: ['Everest', 'stands', 'as', 'a', 'beacon', 'for', 'climbers', 'around', 'the', 'world', '.']
Labels: ['B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

Tokens: ['The', 'path', 'to', 'the', 'summit', 'of', 'K2', 'is', 'fraught', 'with', 'danger', '.']
Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O']

Tokens: ['Many', 'adventurers', 'have', 'dreamed', 'of', 'conquering', 'Kangchenjunga', '.']
Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O']

Tokens: ['At', '8', ',', '516', 'meters', ',', 'Lhotse', 'towers', 'over', 'most', 'other', 'peaks', '.']
Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O']

Tokens: ['The', 'first', 'ascent', 'of', 'Makalu', 'was', 'achieved', 'in', '1955', '.']
Labels: ['O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O']

Tokens: ['Cho', 'Oyu', ',', 'the', 'sixth', '-', 'highest', 'mountain', 'in', 'the', 'world', ',', 'is', 'often', 'considered', 'a', 'good', 'training', 'peak', 'for', 'Eve

Save CoNLL format database in .csv file

In [20]:

formatted_data = []

for sentence_data in sentences_with_labels:
    tokens = sentence_data['Tokens']
    labels = sentence_data['Labels']


    formatted_data.append([tokens, labels])

formatted_df = pd.DataFrame(formatted_data, columns=['Tokens', 'Labels'])

formatted_df.to_csv('dataset_redacted.csv', index=False, header=True)
print(formatted_df)

                                                Tokens  \
0    [Everest, stands, as, a, beacon, for, climbers...   
1    [The, path, to, the, summit, of, K2, is, fraug...   
2    [Many, adventurers, have, dreamed, of, conquer...   
3    [At, 8, ,, 516, meters, ,, Lhotse, towers, ove...   
4    [The, first, ascent, of, Makalu, was, achieved...   
..                                                 ...   
795  [The, icy, slopes, of, Denali, and, the, lush,...   
796  [The, scenic, beauty, of, Elgon, complements, ...   
797  [Annapurna, and, Dhaulagiri, tower, majestical...   
798  [The, cultural, significance, of, Toubkal, dra...   
799  [The, stunning, landscapes, of, Matterhorn, an...   

                                                Labels  
0                [B-LOC, O, O, O, O, O, O, O, O, O, O]  
1             [O, O, O, O, O, O, B-LOC, O, O, O, O, O]  
2                         [O, O, O, O, O, O, B-LOC, O]  
3          [O, O, O, O, O, O, B-LOC, O, O, O, O, O, O]  
4                 